<a href="https://colab.research.google.com/github/Quansight-Labs/uarray/blob/master/notebooks/2018.11.20%20PyData%20Presentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ["A World Where Many Worlds Fit"](https://globalsocialtheory.org/topics/zapatismo/)



[![](https://javiersoriaj.files.wordpress.com/2013/06/575574_431377483625435_324039294_n.jpg)](https://javiersoriaj.wordpress.com/2013/06/09/postales-zapatistas-74-un-mundo-donde-quepan-muchos-mundos/)



* Hardware becoming more hetergenous and parallel
* Data increasing



Need software to target many backends, that is pluggable and allows collaboration. Some existing work in Python land:

* [`opt_einsum`](https://github.com/dgasmith/opt_einsum)
* Extending NumPy dispatching: [NEP 18](http://www.numpy.org/neps/nep-0018-array-function-protocol.html)
* [Tensor Comprehensions](https://facebookresearch.github.io/TensorComprehensions/introduction.html)



Why? So that we can innovate at all levels of stack (hardware, compilation, algorithms, user interface) and share work!



Like thin waist model of Internet Protocal. We working on the "thin waist" that supports common NumPy and SciPy algorithms and can target different hardware (GPU, CPU, FGPA) and software (NumPy, PyTorch, Tensorflow).

* Very preliminary work
* Lot's of fun problems would love collaboration!

# Example

Look at an example using numpy.multiply.outer.  Recall that the outer method of ufuncs takes two arrays a, and b, and creates an array c, with shape a.shape + b.shape (tuple concatenation)

so, for 1-d arrays if c = numpy.multiply.outer(a,b) we will get a 2-d array whose elements are:

$c[i,j] = a[i] * b[j]$

In [0]:
import numpy
numpy.multiply.outer(numpy.arange(5), numpy.arange(10))

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18],
       [ 0,  3,  6,  9, 12, 15, 18, 21, 24, 27],
       [ 0,  4,  8, 12, 16, 20, 24, 28, 32, 36]])

In [0]:

def outer_then_index(a, b):
    return numpy.multiply.outer(a, b)[5]

In [0]:
n = 10

outer_then_index(numpy.arange(n, dtype="float64"), numpy.arange(n, dtype="float64"))

array([ 0.,  5., 10., 15., 20., 25., 30., 35., 40., 45.])

## Issues


* Wasted work: $O(n^2)$ instead of $O(n)$
* Execution tied to API: Requires NumPy-compatible array objects 

## Improved

In [0]:
def outer_then_index_optimized(a, b):
    length_b = b.shape[0] 
    out = numpy.empty((length_b,))
    for i in range(length_b):
        out[i] = a[5] * b[i]
    return out

In [0]:
outer_then_index_optimized(numpy.arange(n, dtype="float64"), numpy.arange(n, dtype="float64"))

array([ 0.,  5., 10., 15., 20., 25., 30., 35., 40., 45.])

I timed these two versions locally on my laptop and found that although at first the improved version is slower, it quickly becomes faster as $n$ increases, due to the differing time complexity:

[![](https://github.com/Quansight-Labs/uarray/raw/master/benchmarks/out.png)](https://github.com/Quansight-Labs/uarray/tree/master/benchmarks)

* No more waisted work (linear instead of quadratic)
* Still requires NumPy-ish object, but only primitive operations

How to automate this translation?

# Theory

In the 1980s Lenore Mullin published ["A Mathematics of Arrays"](https://paperpile.com/app/p/5de098dd-606d-0124-a25d-db5309f99394) based on her work on [APL](https://en.wikipedia.org/wiki/APL_(programming_language).

In it, arrays are defined by:

* Shape (vector of integers)
* Indexing (function from indices to value)

In `outer_then_index_optimized` above, we did the same, determining:

* Shape of result, based on shapes of inputs
* The value of each index of the result

We can use this to perform this translation by hand, then we will look into automating it.

## Hand optimizing the example

### Definitions

I should start by presenting some notation we will use to the derivation that is from the above paper. Here are translations to roughly equivalent NumPy notation:

* $\rho(x)$ = `x.shape`
* $<x, y, z>$ = `numpy.array([x, y, z])`
* $x \psi y$ = `y[x]` where `x` is a tuple of indices
* $x \cdot_{\times} y$ = `numpy.multiply.outer(x, y)`
* $x \downarrow y$ = `y[x:]`
* $x ++ y$ = `numpy.concatenate([x, y])`


And now we can define how outer product and partial indexing work in terms of these.  We start by defining their shapes:

$$
\begin{array}{cc}
\rho \left( x \psi y \right) \equiv  (<0> \psi \rho(x) ) \downarrow \rho\left(y \right)  &  \texttt{(y[x]).shape == y.shape[x.shape[0]:]} \\
\rho \left( x \cdot_{\times} y \right) \equiv \rho(x) ++ \rho(y) & \texttt{(np.multiply.outer(x,y)).shape == x.shape + y.shape}
\end{array}
$$

 And then how to index into them:
 
 $$
\begin{array}{cc}
z \psi (x \psi y) \equiv (x ++ z) \psi y & \texttt{y[x][z] == y[np.concatenate([x,z])]}\\
(j++k) \psi (x \cdot_{\times} y) \equiv j \psi x \times k \psi y & \texttt{np.multiply.outer(x,y)[j+k] == x[j]*y[k] } \\
\end{array}
$$

### Reducing

#### Shape

Assuming that both inputs are vectors and letting their lengthts be $c$ and $d$, we have:


$$
\rho\left(a\right) \equiv<c> \\
\rho\left(b\right) \equiv<d> \\
res \equiv <5> \psi \left(a \cdot_{\times} b\right)
$$


First let's figure out the shape of our result, simply by applying relevent equivalencies:

$$
\rho(res) \equiv \rho  \left(<5> \psi \left(a \cdot_{\times} b\right) \right)  \\
\text{shape of partial index:} \\
\equiv 1 \downarrow \rho  \left(a \cdot_{\times} b\right)\\
\text{shape of outer product:} \\
\equiv 1 \downarrow \left(\rho a  ++ \rho b\right)\\
\text{shapes of inputs} \\
\equiv 1 \downarrow \left(<c>  ++ <d>\right)\\
\text{concat vectors} \\
\equiv 1 \downarrow <c d> \\
\text{drop vectors} \\
\equiv  < d> \\
$$

The result shape is a vector with the length of $b$, which matches our code above.


#### Indexing



Now we can index with the vector $<i>$ and see what the result is:



$$
<i> \psi res \equiv <i> \psi  \left(<5> \psi \left(a \cdot_{\times} b\right) \right)  \\
\text{partial indexing} \\
\equiv <5 \, i> \psi \left(a \cdot_{\times} b\right) \\
\text{indexing outer product} \\
\equiv \left(<5> \psi  a \right) \times \left(<i> \psi  b \right) \\
$$

This indexing expression matches the code we wrote above.

## Automatically optimizing the example

We have started building the [`uarray`](https://github.com/Quansight-Labs/uarray/tree/master/uarray#uarray) that includes a framework to register these types of definitions and reduce expressions:.

In [0]:
!pip install -U uarray==0.4

    100% |████████████████████████████████| 71kB 2.2MB/s 
    100% |████████████████████████████████| 71kB 4.6MB/s 
    100% |████████████████████████████████| 92kB 5.1MB/s 
    100% |████████████████████████████████| 81kB 5.2MB/s 


In [0]:
from uarray import *
import numpy

### Inputs



Let's start by creating two arrays both with dimensionality of 1:

In [0]:
a = with_dims(unbound("a"), 1)
b = with_dims(unbound("b"), 1)
a

Sequence(
    Length(Unbound(variable_name="a")),
    UnaryFunction(
        Scalar(Content(CallUnary(GetItem(Unbound(variable_name="a")), Unbound(variable_name="i2")))),
        Unbound(variable_name="i2"),
    ),
)


### Result

Now we can build up our result given these two arrays

In [0]:

res = Index(vector(5), OuterProduct(multiply, a, b))
res

Index(
    Sequence(Int(1), VectorCallable(Scalar(Int(5)))),
    OuterProduct(
        BinaryFunction(
            Scalar(Multiply(Content(Unbound(variable_name="i0")), Content(Unbound(variable_name="i1")))),
            Unbound(variable_name="i0"),
            Unbound(variable_name="i1"),
        ),
        Sequence(
            Length(Unbound(variable_name="a")),
            UnaryFunction(
                Scalar(Content(CallUnary(GetItem(Unbound(variable_name="a")), Unbound(variable_name="i2")))),
                Unbound(variable_name="i2"),
            ),
        ),
        Sequence(
            Length(Unbound(variable_name="b")),
            UnaryFunction(
                Scalar(Content(CallUnary(GetItem(Unbound(variable_name="b")), Unbound(variable_name="i3")))),
                Unbound(variable_name="i3"),
            ),
        ),
    ),
)


#### Verifying the Result

Now we can call `replace` to do what we did above in an automated way.


*We use the [MatchPy](https://github.com/HPAC/matchpy) library in Python to do this replacement, using pattern matching.*

In [0]:
replaced_res = replace(res)
replaced_res

Sequence(
    Length(Unbound(variable_name="b")),
    UnaryFunction(
        Scalar(
            Multiply(
                Content(CallUnary(GetItem(Unbound(variable_name="a")), Int(5))),
                Content(CallUnary(GetItem(Unbound(variable_name="b")), Unbound(variable_name="i5"))),
            )
        ),
        Unbound(variable_name="i5"),
    ),
)


However, this is still not totally intelligble.

##### Shape

Let's make sure the shape is right. It should be equal to b's shape:

In [0]:
assert replace(Shape(replaced_res)) == replace(Shape(b))

##### Index

Also, let's make sure indexing it gives the right result as well:

In [0]:
i = with_dims(unbound("i"), 0)


expected_index = Scalar(Multiply(
  Content(CallUnary(GetItem(a), Int(5))),
  Content(CallUnary(GetItem(b), Content(i))),
))

assert replace(Index(vector_of(i), replaced_res))  == replace(expected_index)

But how do we actually use these results?

# User Interface

We have started building some interface to build up these expressions and then to turn them into something we can execute.

## Using NumPy Syntax



Starting with a NumPy-ish object that holds an expression tree inside to represent the array.

Here is how we would build the same expression as above:

In [0]:
numpy.multiply.outer(LazyNDArray(a), LazyNDArray(b))[10] 

LazyNDArray(
    Index(
        Sequence(Int(1), VectorCallable(Scalar(Int(10)))),
        OuterProduct(
            BinaryUfunc(np.ufunc(multiply)),
            Sequence(
                Length(Unbound(variable_name="a")),
                UnaryFunction(
                    Scalar(Content(CallUnary(GetItem(Unbound(variable_name="a")), Unbound(variable_name="i2")))),
                    Unbound(variable_name="i2"),
                ),
            ),
            Sequence(
                Length(Unbound(variable_name="b")),
                UnaryFunction(
                    Scalar(Content(CallUnary(GetItem(Unbound(variable_name="b")), Unbound(variable_name="i3")))),
                    Unbound(variable_name="i3"),
                ),
            ),
        ),
    )
)


## Compiling to NumPy code

We also provide an `optimize` decorator that:

* Takes existing function that acceptst and returns NumPy arrays and returns a new function
* Builds up array expression by using the `LazyNDArray`
* Compiles that array expression to a Python AST for reduced code

In [0]:
outer_then_index_auto_optimized =  optimize(1, 1)(outer_then_index)

## Produced Code

If we look at the code it generates, we see it matches semantically our optimized expression above, even though it's much uglier!

In [0]:
print(outer_then_index_auto_optimized.__optimize_steps__['ast_as_source'])



def fn(a, b):
    i_7 = ()
    i_8 = b.shape[0]
    i_3 = ((i_8,) + i_7)
    i_0 = np.empty(i_3)
    i_4 = b.shape[0]
    for i_5 in range(i_4):
        i_6 = i_0[i_5]
        i_1 = 5
        i_2 = a
        i_11 = i_2[i_1]
        i_9 = i_5
        i_10 = b
        i_12 = i_10[i_9]
        i_6 = (i_11 * i_12)
        i_0[i_5] = i_6
    return i_0



# Steppping back

## Goals


* Open and extensible interface
* Focused on needs of community
* Building structure to make it resiliant to change and long lasting


## Next steps



* Expanding the NumPy / SciPi API coverage
* Improving code generation
  * Add lower level backends
  * Adding type sypport for values
  * Make sure it is easy to target high level ops directly to backend
* Making core system more sound
  * More declerative registrations
  * Verify semantics
  * Leverage SymPy


## Interesting Problems

* Algebraic pattern matching / symbolic computing
* Category theoretic approaches
* Friendly registration mechanisms
* Low level optimizations
* Compiler theory


We need your help! Would love to chat about use cases as well.

Thank you!